Digital daydreaming in latent space with CLIP Interogator and Stable Diffusion

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-5a75f273-2b31-6830-0590-67f20ebd0d31)


In [2]:
!pip install diffusers==0.2.4
#!pip install -qq -U diffusers==0.3.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 112 kB 21.9 MB/s 
     |████████████████████████████████| 182 kB 8.8 MB/s 


In [3]:
import subprocess

def setup():
    install_cmds = [
        #['pip', 'install', '-e', 'git+https://github.com/openai/CLIP.git@main#egg=clip'],
        ['pip', 'install', '-e', 'git+https://github.com/MirageML/BLIP.git@main#egg=blip'],#  https://github.com/pharmapsychotic/BLIP.git@lib#egg=blip'],
        ['git', 'clone', 'https://github.com/pharmapsychotic/clip-interrogator.git'],
        ['pip', 'install', 'open_clip_torch', 'ftfy', 'gradio', 'regex', 'tqdm', 'transformers==4.24.0', 'timm', 'fairscale', 'requests']        
    ]
    for cmd in install_cmds:
        print(subprocess.run(cmd, stdout=subprocess.PIPE).stdout.decode('utf-8'))

setup()

import sys
sys.path.append('src/blip')
sys.path.append('src/clip')
sys.path.append('clip-interrogator')

import gradio as gr
from clip_interrogator import Config, Interrogator

ci = Interrogator(Config())

def inference(image, mode):
    image = image.convert('RGB')
    if mode == 'best':
        return ci.interrogate(image)
    elif mode == 'classic':
        return ci.interrogate_classic(image)
    else:
        return ci.interrogate_fast(image)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining blip from git+https://github.com/MirageML/BLIP.git@main#egg=blip
  Cloning https://github.com/MirageML/BLIP.git (to revision main) to ./src/blip
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for fairscale: filename=fairscale-0.4.4-py3-none-any.whl size=292864 sha256=e8d2e74c1a938e976255f85c2dae89ef2d9af27244499b12fbdac00411d70b40
  Stored in directory: /root/.cache/pip/wheels/7e/01/9d/ea7ac6a01eb4044323d028f92fe2ac26c36157ad01b2b907c1
Successfully built fairscale
  Running setup.py develop for blip

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

  0%|          | 0.00/1.66G [00:00<?, ?B/s]

load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth
Loading CLIP model...


100%|███████████████████████████████████████| 933M/933M [00:22<00:00, 42.0MiB/s]
Preprocessing trendings: 100%|██████████| 1/1 [00:00<00:00,  9.51it/s]

Loaded CLIP model and data in 211.00 seconds.


In [4]:
!mkdir spun
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/stub_materials/figor.png /content/st.png #your seed image
!cp /content/drive/MyDrive/stub_materials/image_to_image.py /content/

Mounted at /content/drive


In [7]:
# don't forget:
# huggingface-cli login
# I always keep my token # here

import os
import argparse
import torch
import numpy as np
from diffusers import StableDiffusionPipeline
from PIL import Image
from io import BytesIO
from image_to_image import StableDiffusionImg2ImgPipeline, preprocess
#from diffusers.models import AutoencoderKL
class FakeSafety():
    def __call__(self, clip_input, images):
        return (images, False)
from torch import autocast
rond=np.random.randint(7,100000000)
generator = torch.Generator("cuda").manual_seed(rond)
print( "random seed " + str(rond))
device = "cuda"
#vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse")
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    #vae=vae,
    revision="fp16", 
    torch_dtype=torch.float16,
    use_auth_token=True
).to(device)

pipe.safety_checker = FakeSafety()
file1 = open("dialog1.txt", "a")
for xn in range(0,250,1):
  im= Image.open("st.png").convert("RGB")
  im=im.resize((512,512))
  cliprompt=ci.interrogate_fast(im)
  print(cliprompt)
  file1.write(cliprompt+"\n")
  init_image = preprocess(im)
  with autocast("cuda"):
    image = pipe(cliprompt, init_image=init_image, strength=0.65, guidance_scale=7.0, num_inference_steps=20, generator=generator)["sample"][0]  # image
  image.save("st.png")
  image.save("spun2/"+str(xn).zfill(4)+".png")
  os.system("bash 1julien.sh st.png zi")
file1.close()
!cp dialog1.txt spun2/
!zip -r 11_29_22_spun2.zip spun2/ 
!cp 11_29_22_spun2.zip /content/drive/MyDrive/product

random seed 39984350


Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

100%|██████████| 55/55 [00:00<00:00, 114.17it/s]


a moss covered area in the middle of a forest, inside an otherworldly planet, neil blomkamp film landscape, inside an alien jungle, ruined alien structures, alien habitat, lush alien landscape, futuristic estructures in ruins, planet overgrown, strange alien forest, frame from prometheus movie, stunning alien landscape, humid alien jungle, surreal abandoned buildings, amazing alien landscape


0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

100%|██████████| 55/55 [00:00<00:00, 219.15it/s]


a painting of an owl with colorful eyes, epic shamanic dmt art, android jones and chris dyer, patterned visionary art, psytrance artwork, visionary art style, psychedelic trippy visionary art, intricate artwork. neon eyes, portrait of a geometric owl, highly detailed visionary art, dmt entity ; lsd art, shamanic poster lsd art, intricate psychedelic patterns


0it [00:00, ?it/s]

  adding: spun2/ (stored 0%)
  adding: spun2/0188.png (deflated 0%)
  adding: spun2/0042.png (deflated 0%)
  adding: spun2/0125.png (deflated 0%)
  adding: spun2/0206.png (deflated 0%)
  adding: spun2/0087.png (deflated 0%)
  adding: spun2/0168.png (deflated 0%)
  adding: spun2/0096.png (deflated 0%)
  adding: spun2/0038.png (deflated 0%)
  adding: spun2/0057.png (deflated 0%)
  adding: spun2/0163.png (deflated 0%)
  adding: spun2/0061.png (deflated 0%)
  adding: spun2/0245.png (deflated 0%)
  adding: spun2/0016.png (deflated 0%)
  adding: spun2/0055.png (deflated 0%)
  adding: spun2/0015.png (deflated 0%)
  adding: spun2/0085.png (deflated 0%)
  adding: spun2/0159.png (deflated 0%)
  adding: spun2/0010.png (deflated 0%)
  adding: spun2/0201.png (deflated 0%)
  adding: spun2/0093.png (deflated 0%)
  adding: spun2/0218.png (deflated 0%)
  adding: spun2/0080.png (deflated 0%)
  adding: spun2/0233.png (deflated 0%)
  adding: spun2/0077.png (deflated 0%)
  adding: spun2/0017.png (deflated 

In [ ]:
from google.colab import runtime
runtime.unassign()